<a href="https://colab.research.google.com/github/SaqlainHussainShah/Image-classificaiton/blob/master/Basic_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from PIL import Image
import json 
import cv2

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path='/content/drive/My Drive/gen_img_data.json'


In [0]:
with open(path, 'r') as f:
  datastore = json.load(f)

In [120]:
print(len(datastore))

23702


In [0]:
df_1=pd.DataFrame()

In [122]:
df_1.head()

""


In [0]:
# df_1.loc[5, '2'] = 23

In [124]:
df_1.head()

""


In [0]:
def get_values_class(json_data):
  labels=[]
  pixel_values=[]
  
  for i in range(len(json_data)):
    
    for name, value in json_data[i].items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
      labels.append(name)
      # print(name)

      for name, value in value.items():
        pixel_values.append(value)
        # print(len(value))
        # print(value)

  return (pixel_values, labels)

In [0]:
X,y=get_values_class(datastore)

In [127]:
len(X)

44704

In [128]:
for i in range(len(X)):
  for j in range(len(X[i])):
    pixel=X[i][j]
    k=str(j)
    df_1.loc[i, k] = pixel

KeyboardInterrupt: ignored

In [129]:
len(df_1)

15431

In [0]:
X_data = np.array(df_1.iloc[:, :])

In [0]:
Y_data=y[:15431]

In [132]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y_data)


LabelEncoder()

In [0]:
y_transform=le.transform(Y_data)
from tensorflow.keras import utils as np_utils
Y_train = np_utils.to_categorical(y_transform, num_classes = 6)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
trainX, testX, trainY, testY = train_test_split(X_data,Y_train, test_size=0.2, random_state=42)

In [158]:
trainX.shape

(12344, 1024)

In [0]:
rows, cols = 32, 32 

train_data = trainX.reshape(trainX.shape[0], rows, cols, 1)
test_data = testX.reshape(testX.shape[0], rows, cols, 1)

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import keras 

from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical 

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization

In [0]:
train_data /= 255.0
test_data /= 255.0

In [0]:
batch_size = 256
epochs = 10
input_shape = (rows, cols, 1)

In [0]:
def baseline_model():

  model = Sequential()
#convolutional layer with rectified linear unit activation
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(32,32,1)))
  #32 convolution filters used each of size 3x3
  #again
  model.add(Conv2D(64, (3, 3), activation='relu'))
  #64 convolution filters used each of size 3x3
  #choose the best features via pooling
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #randomly turn neurons on and off to improve convergence
  model.add(Dropout(0.25))
  #flatten since too many dimensions, we only want a classification output
  model.add(Flatten())
  #fully connected to get all relevant data
  model.add(Dense(128, activation='relu'))
  #one more dropout for convergence' sake :) 
  model.add(Dropout(0.5))
  #output a softmax to squash the matrix into output probabilities
  model.add(Dense(6, activation='softmax'))
  return model

In [0]:
model = baseline_model()
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [164]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 12544)             0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               1605760   
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)             

In [165]:
history = model.fit(train_data, trainY,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_data, testY))
          

Train on 12344 samples, validate on 3087 samples
Epoch 1/10
12344/12344 [==============================] - 3s 209us/step - loss: 1.6873 - acc: 0.3763 - val_loss: 1.3586 - val_acc: 0.3816
Epoch 2/10
12344/12344 [==============================] - 1s 80us/step - loss: 1.3911 - acc: 0.3837 - val_loss: 1.3577 - val_acc: 0.3816
Epoch 3/10
12344/12344 [==============================] - 1s 80us/step - loss: 1.3901 - acc: 0.3826 - val_loss: 1.3584 - val_acc: 0.4017
Epoch 4/10
12344/12344 [==============================] - 1s 78us/step - loss: 1.3877 - acc: 0.3884 - val_loss: 1.3601 - val_acc: 0.3816
Epoch 5/10
12344/12344 [==============================] - 1s 78us/step - loss: 1.3842 - acc: 0.3820 - val_loss: 1.3615 - val_acc: 0.4017
Epoch 6/10
12344/12344 [==============================] - 1s 78us/step - loss: 1.3860 - acc: 0.3829 - val_loss: 1.3598 - val_acc: 0.3816
Epoch 7/10
12344/12344 [==============================] - 1s 78us/step - loss: 1.3860 - acc: 0.3863 - val_loss: 1.3584 - val_acc